In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [27]:
df = pd.read_csv("ILPD.csv")
df

,Age,Gender,TB,DB,AP,SGPT,SGOT,TP,ALB,A/G,Target
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,2
579,40,Male,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,Male,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,Male,1.3,0.5,184,29,32,6.8,3.4,1.00,1


# Encoding Gender column

In [28]:
from sklearn.preprocessing import LabelEncoder

label_encoder_x = LabelEncoder()
df['Gender'] = label_encoder_x.fit_transform(df[['Gender']])
df

C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,Age,Gender,TB,DB,AP,SGPT,SGOT,TP,ALB,A/G,Target
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,1,0.5,0.1,500,20,34,5.9,1.6,0.37,2
579,40,1,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,1,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,1,1.3,0.5,184,29,32,6.8,3.4,1.00,1


# Checking for NULL values

In [29]:
df.isnull().values.any()

True

In [30]:
df.isnull().sum().sum()

4

In [31]:
df[df.isnull().any(axis=1)]

,Age,Gender,TB,DB,AP,SGPT,SGOT,TP,ALB,A/G,Target
209,45,0,0.9,0.3,189,23,33,6.6,3.9,NaN,1
241,51,1,0.8,0.2,230,24,46,6.5,3.1,NaN,1
253,35,0,0.6,0.2,180,12,15,5.2,2.7,NaN,2
312,27,1,1.3,0.6,106,25,54,8.5,4.8,NaN,2


# Handling missing data
## 1. Replacing missing data with the mean value

In [32]:
from sklearn.impute import SimpleImputer

df1 = df.copy()
imputer = SimpleImputer(missing_values = np.nan, strategy ='mean')

# Fitting the data to the imputer object
imputer = imputer.fit(df1[['A/G']])  

# Imputing the data   
df1['A/G'] = imputer.transform(df1[['A/G']])  

In [33]:
df1.isnull().any()

Age       False
Gender    False
TB        False
DB        False
AP        False
SGPT      False
SGOT      False
TP        False
ALB       False
A/G       False
Target    False
dtype: bool

In [34]:
df1.iloc[[209,241,253,312],:]

,Age,Gender,TB,DB,AP,SGPT,SGOT,TP,ALB,A/G,Target
209,45,0,0.9,0.3,189,23,33,6.6,3.9,0.947064,1
241,51,1,0.8,0.2,230,24,46,6.5,3.1,0.947064,1
253,35,0,0.6,0.2,180,12,15,5.2,2.7,0.947064,2
312,27,1,1.3,0.6,106,25,54,8.5,4.8,0.947064,2


## 2. Handling Missing data by MultiVariate Imputation

In [35]:
df2 = df.copy()

In [36]:
cont_feats = [col for col in df2.columns if df2[col].dtype != object]

In [37]:
iter_imputer = IterativeImputer(random_state=42)
iter_imputed = iter_imputer.fit_transform(df2[cont_feats])
df2 = pd.DataFrame(iter_imputed, columns=cont_feats)

In [38]:
df2.iloc[[209,241,253,312],:]

,Age,Gender,TB,DB,AP,SGPT,SGOT,TP,ALB,A/G,Target
209,45.0,0.0,0.9,0.3,189.0,23.0,33.0,6.6,3.9,1.309887,1.0
241,51.0,1.0,0.8,0.2,230.0,24.0,46.0,6.5,3.1,0.918076,1.0
253,35.0,0.0,0.6,0.2,180.0,12.0,15.0,5.2,2.7,0.993674,2.0
312,27.0,1.0,1.3,0.6,106.0,25.0,54.0,8.5,4.8,1.373912,2.0
